## Multiuser RAG

In [24]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser 
from langchain_core.prompts import  ChatPromptTemplate ,MessagesPlaceholder
from langchain.memory import FileChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from pathlib import Path

model = ChatOpenAI(temperature = 0 , model ="gpt-3.5-turbo")

In [4]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [5]:
history_dir= Path("./message_history")
history_dir.mkdir(parents  = True , exist_ok=True)

In [6]:
def get_session_history(session_id:str) -> FileChatMessageHistory:
    session_history_path = history_dir / f"{session_id}.json"
    return FileChatMessageHistory(str(session_history_path))

In [7]:
conv_chain = RunnableWithMessageHistory(
    chain , get_session_history ,  input_messages_key="input",
    history_messages_key="chat_history"
)

In [8]:
query = "What is AI"
session_id = "user001"

In [9]:
conv_chain.invoke({'input':query},
                 {'configurable':{'session_id':session_id}})

Parent run 9956faa5-445b-441e-a305-dd9181fcbf31 not found for run f58e1d40-ecde-46f1-bc64-d5d0dfdbea28. Treating as a root run.


'AI, or artificial intelligence, refers to the simulation of human intelligence processes by machines, such as learning, reasoning, and self-correction.'

In [10]:
conv_chain.invoke({'input':"what is my previous question ?"},
                 {'configurable':{'session_id':session_id}})

Parent run 90133505-db62-4f9a-8884-6697d7404812 not found for run 5241f635-9da5-4d82-9a84-dc761d1e5dce. Treating as a root run.


'Your previous question was "What is AI?"'

In [11]:
conv_chain.invoke({'input':"How it is different than Rule  based system ?"},
                 {'configurable':{'session_id':session_id}})

Parent run 798db8b0-36d4-42d3-981e-972f0241e123 not found for run 73489b27-44db-4db0-8d4c-55ec8deeabe0. Treating as a root run.


'AI involves the simulation of human intelligence processes by machines, while rule-based systems rely on predefined rules and logic to make decisions or perform tasks.'

# RAG

In [27]:
from langchain.vectorstores import  FAISS
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.output_parser import StrOutputParser 
from langchain.memory import FileChatMessageHistory
from pathlib import Path

import os
os.environ["OPENAI_API_KEY"] = ""

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [15]:
# Loading already created index
vectorstore=FAISS.load_local("index" , embeddings=OpenAIEmbeddings() ,  allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [16]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [17]:
history_dir= Path("./message_history")
history_dir.mkdir(parents  = True , exist_ok=True)

def get_session_history(session_id:str) -> FileChatMessageHistory:
    session_history_path = history_dir / f"{session_id}.json"
    return FileChatMessageHistory(str(session_history_path))

In [18]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [19]:
conversational_rag_chain.invoke(
    {"input": "What is machine learning ?"},
    config={
        "configurable": {"session_id": "user001"}
    },  
)["answer"]

Parent run 8ed2bd47-1767-4094-8a22-0eaebc7cb3d4 not found for run 046749b3-4b73-4ed2-b488-15d47fe32d39. Treating as a root run.


'Machine learning is a transformative branch of artificial intelligence that empowers computers to learn from and make decisions based on data. Unlike traditional programming, where explicit instructions dictate the outcome, ML algorithms identify patterns and infer rules directly from the data they process, improving their performance with experience over time.'

In [20]:
conversational_rag_chain.invoke(
    {"input": "What is my previous question ?"},
    config={
        "configurable": {"session_id": "user001"}
    },  
)["answer"]

Parent run d44a50cf-450b-49ea-a218-344d3791af13 not found for run bba275aa-9ae9-4051-b0f0-813df9245dab. Treating as a root run.


'Your previous question was "What is machine learning?"'

In [21]:
conversational_rag_chain.invoke(
    {"input": "How it is different than deep learning. Give answer in one line."},
    config={
        "configurable": {"session_id": "user001"}
    },  
)["answer"]

Parent run b677d705-2158-46d5-8ea0-86312d64c479 not found for run 19194a3e-4d34-40c1-97b6-4e736379b94f. Treating as a root run.


'Deep learning is a subset of machine learning that uses neural networks with many layers to model complex patterns in large datasets.'

In [22]:
conversational_rag_chain.invoke(
    {"input": "what  is supervised learning"},
    config={
        "configurable": {"session_id": "user002"}
    },  
)["answer"]

Parent run ba4e8e8f-47a2-4acf-a272-2d9a7f8be1df not found for run 6cd7cffa-0ddf-4ab5-8750-ce66447ecf94. Treating as a root run.


'Supervised learning involves training models on labeled data to make predictions or classifications. It includes techniques like regression for continuous outcomes and classification for discrete categories. Common algorithms in supervised learning include linear regression, decision trees, random forests, support vector machines, and neural networks.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "what is my previously asked question ?"},
    config={
        "configurable": {"session_id": "user002"}
    },  
)["answer"]

Parent run af4c03f6-0e7c-4a80-9f26-d340a234c0ef not found for run 04a82388-32d4-4939-b2db-4897da32b2ca. Treating as a root run.


'Your previously asked question was about supervised learning, which involves training models on labeled data to make predictions or classifications. It includes techniques like regression for continuous outcomes and classification for discrete categories. Common algorithms in supervised learning include linear regression, decision trees, random forests, support vector machines, and neural networks.'

In [23]:
conversational_rag_chain.invoke(
    {"input": "what is my previously asked question ?"},
    config={
        "configurable": {"session_id": "user001"}
    },  
)["answer"]

Parent run ca19a72b-51e5-445d-911c-f6b1eb3c1c9a not found for run c263e5a3-07b9-4243-8597-4e3a7c21e6d6. Treating as a root run.


'Your previous question was "How it is different than deep learning. Give answer in one line."'